## Poisuelle Flow Using LBM:

In the lattice boltzmann model, one is to solve for the probability distribution function of the "particles". 
I'll be trying to solve the model for a 2D poisuelle flow in a pipe.
The grid used in the problems would be a $D_2Q_9$ type of grid with 9 discretized velocities with different weights.

The calculations on a lattice boltzmann solver are performed in two steps: 1.) the collision 2.) streaming

## Connecting the microscopic quantities to macroscopic:

The probability distribution function of a lattice point is given by $f(\vec{x},\vec{\xi},t)$. Where $\vec{x}$ is the "molecular" position and $\vec{\xi}$ is the microscopic velocity. The microscopic velocity here is discretized in 9 directions for the $D_2Q_9$ cell with weights that normalize the weighted probability density.
The function $f$ gives the "probability" of a particle at a given position and time, with a microscopic velocity would be moving in the direction $\vec{\xi}$ due to kintetic effects.

The dicretized PDF is given by: $f_{\alpha}(\vec{x},t)$
where alpha is the index of the 9 directions it can take in the $D_2Q_9$ cell.

The macroscopic quanities are given by:

Density:
$$ \rho(\vec{x},t) = \sum_{\alpha} f_{\alpha}(\vec{x},t)$$

Momentum per unit volume:

$$ \rho(\vec{x},t)\vec{u}(\vec{x},t) = \sum_{\alpha} f_{\alpha}(\vec{x},t)\vec{c}_{\alpha}$$

Here, $\vec{c}_{\alpha} $ is the discrete velocity vector set

Pressure for a lattice point can be calculated using: $P = \rho c_s^2 $

## The Lattice Boltzman Equation:

The lattice boltzmann equation is given by:

$$f_{\alpha}(\vec{x}+\vec{c}_{\alpha}\Delta t,t+\Delta t) - f_{\alpha}(\vec{x},t) = -\frac{(f_{\alpha}(\vec{x},t))- f_{\alpha}^{eq}(\vec{x},t)}{\tau/\Delta t}$$

Here, the $f_{\alpha}^{eq}(\vec{x},t)$ comes from the maxwell boltzmann distribution, which is given by:
$$ f_{\alpha}^{eq}(\vec{x},t) = w_\alpha \rho \left(1+ \frac{\vec{u}\cdot \vec{c}_\alpha}{c_s^2}+\frac{(\vec{u}\cdot \vec{c}_\alpha)^2}{2c_s^4}-\frac{\vec{u}\cdot \vec{u}}{2c_s^2}\right)$$

Here:
\
\begin{align*}
\rho & \rightarrow \text{macroscopic density} (\vec{x},t) \\
\vec{u} & \rightarrow \text{macroscopic velocity} (\vec{x},t)\\
w_\alpha & \rightarrow \text{weights given to microscopic velocity} (\vec{x},t)\\
c_\alpha & \rightarrow \text{discrete velocity} (\vec{x},t)\\
c_s & \rightarrow \text{speed of sound} (\vec{x},t)
\ \text{where: } c_s^2 = \frac{1}{3}\left(\frac{\Delta x}{\Delta t}\right)^2
\end{align*}
\

The pressure is now directly related to density as $P = \rho c_s^2$

The $f_{\alpha}^{eq}(\vec{x},t)$ satisfies the normal PDF conditions:
$$ \sum_\alpha f_{\alpha}^{eq}(\vec{x},t) = \rho(\vec{x},t) $$
and 
$$ \sum_\alpha f_{\alpha}^{eq}(\vec{x},t)\;\vec{c}_\alpha = \rho\;\vec{u}(\vec{x},t) $$


## Steps for Solving:

### 1.) Collision:

$$ f_{\alpha}^\star(\vec{x},t) = f_{\alpha}(\vec{x},t) - \frac{\Delta t}{\tau}\left(f_{\alpha}(\vec{x},t)-f_{\alpha}^{eq}(\vec{x},t)\right)$$

The collision equation is modified in the presence of an external force.
( See section "Gno's forcing Scheme for reference)

### 2.) Streaming:

$$ f_{\alpha}(\vec{x}+\vec{c}_{\alpha}\Delta t,t+\Delta t) = f_{\alpha}^\star(\vec{x},t) $$

This is the step where you basically assign the distribution function to the subsequent cell.

### 3.) Application of boundary conditions

#### Bounceback Boundary Condition:

The existance of a wall is resolved by the use of half-way bounceback boundary condition.
$$ f_\alpha (x_f, t+\Delta t) =f_{\alpha '}^* (x_f, t+\Delta t) $$

The $\alpha '$ is the opposite direction of the original $\alpha$ direction.
So, for direction 4,7 and 8, the complementary directions are 2,5 and 6 respectively.

## Outflow and Inflow Boundary condition

### Outflow boundary condition is enforced using the following way:

distributions 6, 3 and 7 for the last column of nodes is taken to be the distribution of the previous (second last) column:
pseudocode:

### Streaming and Bounceback Simplified using BounceBack:

Streaming and bounceback using numpy

### Inlet Boundary Condtion:

The inlet boundary condition used here is the equilibrium scheme dirichlet velocity boundary condition.

Under the equilibrium scheme, the wall densities are calculated for the left wall using:

$$ \rho_w = \frac{c}{c-u_{w,x}}[f_0+f_2+f_4+2(f_3+f_6+f_7)]$$

And for the right wall (Outlet), one "may" use:

$$ \rho_w = \frac{c}{c+u_{w,x}}[f_0+f_2+f_4+2(f_1+f_5+f_8)]$$

And using the equilibrium scheme, the boundary distribution set could be just replaced by:
$$ f_i(x_b,t) = f_i^{eq}(\rho_w,u_w) $$

### Steps for Implementation:

1.) Apply Outflow BC.

2.) Compute machroscopic density and momenta.

3.) Apply the dirichlet velocity boundary condition.

4.) Compute feq.

5.) Perform collision using the BGK Operator.

6.) Apply bounce-back BC.

7.) Stream velocity along ci.

8.) Advance time loop.